In [1]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
from collections import namedtuple
import folium
import random
from datetime import datetime
from folium import plugins
import pandas as pd

ModuleNotFoundError: No module named 'folium'

In [ ]:
def create_data_model(jsfile):
    """Stores the data for the problem."""
    #set code for times that distance and time matrix are given
    coded=0
    codet=0
    data = {}
    data['points']=[[jsfile['points'][i]['latitude'],jsfile['points'][i]['longitude']] for i in range(len(jsfile['points']))]
    if jsfile['constraint']:
        if jsfile['constraint']['distance']:
            data['distance_matrix']=[[value for key,value in jsfile['constraint']['distance'][0][i].items()] for i in range(1,len(jsfile['constraint']['distance'][0])+1)]
            coded=1

        if jsfile['constraint']['time']:
            data['time_matrix']=[[value for key,value in jsfile['constraint']['time'][0][i].items()] for i in range(1,len(jsfile['constraint']['time'][0])+1)]
            codet=1

    data['codes']=[coded,codet]
    data['cost_matrix']=[[value for key,value in jsfile['cost'][0][i].items()] for i in range(1,len(jsfile['cost'][0])+1)]
    data['num_vehicles'] = len(jsfile['cars'])
#     data['start']=[0 for i in range(len(jsfile['cars']))]
    data['start'] = [24, 26]
#     data['end']=data['start']
    data['end'] =[25, 27]
    data['depot'] = 0
    try:
        data['demands']=[i['amount'] for i in jsfile['points']]
        data['vehicle_capacities']=[i['capacity'] for i in jsfile['cars']]
    except:
        pass

    return data

In [ ]:
jsf={
"cars":[{"id":1,"capacity":10},{'id':2,'capacity':12}],
"constraint":{'distance':[{1:{1:0,2:26,3:6,4:5},2:{1:26,2:0,3:16,4:4},3:{1:6,2:16,3:0,4:11},4:{1:5,2:4,3:11,4:0}}],'time':[]},
"points":[{"id":1,"amount":5,"latitude":35.806900, "longitude":51.429000}, {"id":2,"amount":6,"latitude":35.699760, "longitude":51.339740},{'id':3,'amount':1,'latitude':35.732095,'longitude':51.482965},{'id':4,'amount':2,'latitude':35.777073,'longitude': 51.347319}],
"cost":[{1:{1:0,2:100,3:6,4:1},2:{1:100, 2:0,3:9,4:5},3:{1:6,2:9,3:0,4:70},4:{1:1,2:5,3:70,4:0}}]
}
data = create_data_model(jsf)
data

In [ ]:
def get_routes(manager, routing, solution, num_routes):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(num_routes):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    #print(routes)
    return routes

def main(data):
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['cost_matrix']), data['num_vehicles'], data['start'],data['end'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    # Create and register a transit callback.
    def cost_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['cost_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    max_int= 2 * 60 * 60
    # Add Cost constraint.
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        max_int,  # vehicle maximum travel distance
        False,  # start cumul to zero
        'Cost')
    cost_dimension = routing.GetDimensionOrDie('Cost')
    cost_dimension.SetGlobalSpanCostCoefficient(1000)

    #Add Distance constraint.
    if data['codes'][0]:
        def distance_callback(from_index,to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        distance_callback_index=routing.RegisterTransitCallback(distance_callback)
        routing.AddDimension(
            distance_callback_index,
            0,
            max_int,
            False,
            'Distance')
        distance_dimension=routing.GetDimensionOrDie('Distance')

    #Add Time constraint.
    if data['codes'][1]:
        def time_callback(from_index,to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['time_matrix'][from_node][to_node]

        time_callback_index=routing.RegisterTransitCallback(time_callback)
        routing.AddDimension(
            time_callback_index,
            0,
            max_int,
            False,
            'Time')
        time_dimension=routing.GetDimensionOrDie('Time')

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # Setting first solution heuristic.
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC)
    #Setting local search
    #search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

    search_parameters.solution_limit = 1000000
    search_parameters.log_search = True
    search_parameters.time_limit.seconds = 30
    #print(search_parameters)
    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
        routes = get_routes(manager, routing, assignment, data['num_vehicles'])
        return routes
    else:
        print('No solution found')


In [ ]:
route=main(data)
route

In [ ]:
df = pd.read_csv("data-bil.csv", header=0) 
df.head()

In [2]:
df = df[df['type']!='استرابورد']
df = df[df['lat'].notnull()]
df = df[(df['start'] < '03-18')]
df = df[df['end'] > '03-18']
df = df.reset_index(drop=True)
len(df)

NameError: name 'df' is not defined

In [ ]:
# df.loc[df.shape[0]] = ['xxx1', 51.372656 ,35.693632 , 'Carriot_1', '03-17', '04-15']
# df.loc[df.shape[0]] = ['xxx2', 51.372656 ,35.693632 , 'Carriot_2', '03-17', '04-15']
# df.loc[df.shape[0]] = ['xxx3', 51.372656 ,35.693632 , 'Carriot_2', '03-17', '04-15']
df.loc[len(df)] = ['شروع' ,51.55529868248729 ,35.73176754573533, 'start', '03-17', '04-15']
df.loc[len(df)] = ['پایان', 51.55529868248729 ,35.73176754573533 , 'end', '03-17', '04-15']
df.loc[len(df)] = ['شروع' ,51.358716666666666,35.642963333333334 , 'start', '03-17', '04-15']
df.loc[len(df)] = ['پایان' ,51.358716666666666,35.642963333333334, 'end', '03-17', '04-15']
df.head()

In [ ]:
df

In [ ]:
#df = df.iloc[0:25].reset_index(drop=True)
# df1 = df.iloc[0:25].reset_index(drop=True)

In [ ]:
#points = ";".join((df['lat'].astype(str) + "," + df['lon'].astype(str)).to_numpy())

In [ ]:
curbs = ";".join(['curb'] * 2)

In [ ]:
#URL = "https://api.mapbox.com/directions-matrix/v1/mapbox/driving/" + points + "?approaches=" + curbs + "&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"

In [ ]:
import requests
#response = requests.get(URL)

In [ ]:
import json
#json_response = json.loads(response.content)
#cost_matrix = json_response['durations']

In [ ]:
df.reset_index(drop=True)
df1=df.iloc[0:int(len(df)/2)].reset_index()
df2=df.iloc[int(len(df)/2):len(df)].reset_index()

In [ ]:
#if (len(df)%2 == 1):
#    df.loc[df.shape[0]] = ['yyyy', '51.372656' ,'35.693632' , 'Carriot_x2']

df.head()
print (len(df))

In [ ]:
df1=df.iloc[0:int(len(df)/4)].reset_index()
df2=df.iloc[int(len(df)/4):int(len(df)/2)].reset_index()
df3=df.iloc[int(len(df)/2):int(len(df)*3/4)].reset_index()
df4=df.iloc[int(len(df)*3/4):len(df)].reset_index()

df_GOD12=df.iloc[0:int(len(df)/2)].reset_index()
df_GOD34=df.iloc[int(len(df)/2):len(df)].reset_index()
df_GOD13=(pd.concat([df1,df3],ignore_index=True)).drop(columns=['index']).reset_index()
df_GOD24=(pd.concat([df2,df4],ignore_index=True)).drop(columns=['index']).reset_index()
df_GOD14=(pd.concat([df1,df4],ignore_index=True)).drop(columns=['index']).reset_index()
df_GOD23=(pd.concat([df2,df3],ignore_index=True)).drop(columns=['index']).reset_index()

print(len(df1))
print(len(df_GOD13))
print(len(df2))
print(len(df3))
print(len(df4))

In [ ]:
print(len(df_GOD14))
df_GOD14.head()

In [ ]:
print(len(df_GOD12))
df_GOD12.head()

In [ ]:
print (URL13)

In [ ]:
points12= ";".join((df_GOD12['lat'].astype(str) + "," + df_GOD12['lon'].astype(str)).to_numpy())
points34= ";".join((df_GOD34['lat'].astype(str) + "," + df_GOD34['lon'].astype(str)).to_numpy())
points13= ";".join((df_GOD13['lat'].astype(str) + "," + df_GOD13['lon'].astype(str)).to_numpy())
points24= ";".join((df_GOD24['lat'].astype(str) + "," + df_GOD24['lon'].astype(str)).to_numpy())
points14= ";".join((df_GOD14['lat'].astype(str) + "," + df_GOD14['lon'].astype(str)).to_numpy())
points23= ";".join((df_GOD23['lat'].astype(str) + "," + df_GOD23['lon'].astype(str)).to_numpy())

points1=";".join((df1['lat'].astype(str) + "," + df1['lon'].astype(str)).to_numpy())
points2=";".join((df2['lat'].astype(str) + "," + df2['lon'].astype(str)).to_numpy())
points3=";".join((df3['lat'].astype(str) + "," + df3['lon'].astype(str)).to_numpy())
points4=";".join((df4['lat'].astype(str) + "," + df4['lon'].astype(str)).to_numpy())

curbs1 = ";".join(['curb'] * len(df_GOD12))
curbs2 = ";".join(['curb'] * len(df1))

In [ ]:
len(df_GOD12)

In [ ]:
URL1 ="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points12+"?approaches="+curbs1+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"
URL2 ="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points34+"?approaches="+curbs1+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"
URL13="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points13+"?approaches="+";".join(['curb'] * 14)+"&sources=0;1;2;3;4;5;6"+"&destinations=7;8;9;10;11;12;13"+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"
URL31="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points13+"?approaches="+";".join(['curb'] * 14)+"&sources=7;8;9;10;11;12;13"+"&destinations=0;1;2;3;4;5;6"+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"
URL24="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points24+"?approaches="+";".join(['curb'] * 14)+"&sources=0;1;2;3;4;5;6"+"&destinations=7;8;9;10;11;12;13"+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"
URL42="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points24+"?approaches="+";".join(['curb'] * 14)+"&sources=7;8;9;10;11;12;13"+"&destinations=0;1;2;3;4;5;6"+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"
URL14="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points14+"?approaches="+";".join(['curb'] * 14)+"&sources=0;1;2;3;4;5;6"+"&destinations=7;8;9;10;11;12;13"+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"
URL41="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points14+"?approaches="+";".join(['curb'] * 14)+"&sources=7;8;9;10;11;12;13"+"&destinations=0;1;2;3;4;5;6"+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"
URL23="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points23+"?approaches="+";".join(['curb'] * 14)+"&sources=0;1;2;3;4;5;6"+"&destinations=7;8;9;10;11;12;13"+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"
URL32="https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"+points23+"?approaches="+";".join(['curb'] * 14)+"&sources=7;8;9;10;11;12;13"+"&destinations=0;1;2;3;4;5;6"+"&access_token=pk.eyJ1IjoiYWJvbGZhemx0YWhlcmkiLCJhIjoiY2p6OXhudmt0MDJ2ZzNtcXM0Z25wMnllaSJ9.HCX4MlHSM1mTXvod27ddWw"

In [ ]:
response1  = requests.get(URL1)
response2  = requests.get(URL2)
response13 = requests.get(URL13)
response31 = requests.get(URL31)
response24 = requests.get(URL24)
response42 = requests.get(URL42)
response23 = requests.get(URL23)
response32 = requests.get(URL32)
response14 = requests.get(URL14)
response41 = requests.get(URL41)

json_response1  = json.loads(response1.content)
json_response2  = json.loads(response2.content)
json_response13 = json.loads(response13.content)
json_response31 = json.loads(response31.content)
json_response24 = json.loads(response24.content)
json_response42 = json.loads(response42.content)
json_response23 = json.loads(response23.content)
json_response32 = json.loads(response32.content)
json_response14 = json.loads(response14.content)
json_response41 = json.loads(response41.content)

cost_matrix1  = np.array(json_response1['durations'])
cost_matrix2  = np.array(json_response2['durations'])
cost_matrix13 = np.array(json_response13['durations'])
cost_matrix31 = np.array(json_response31['durations'])
cost_matrix24 = np.array(json_response24['durations'])
cost_matrix42 = np.array(json_response42['durations'])
cost_matrix23 = np.array(json_response23['durations'])
cost_matrix32 = np.array(json_response32['durations'])
cost_matrix14= np.array(json_response14['durations'])
cost_matrix41= np.array(json_response41['durations'])

In [ ]:
json_response13

In [ ]:
x1 = np.concatenate((cost_matrix13 ,cost_matrix14) , axis=1)
x2 =  np.concatenate((cost_matrix23 ,cost_matrix24) , axis=1)
x12 = np.concatenate((x1 , x2) , axis=0)

x3 = np.concatenate((cost_matrix31 ,cost_matrix32) , axis=1)
x4 = np.concatenate((cost_matrix41 ,cost_matrix42) , axis=1)
x34 = np.concatenate((x3 , x4) , axis=0)

result_up = np.concatenate((cost_matrix1,x12),axis=1)
result_do = np.concatenate((x34,cost_matrix2),axis=1)
result = np.concatenate((result_up , result_do) , axis=0)

In [ ]:
print(result)

In [ ]:
import random
points = []
counter = 1
index_to_id = {}
for index, row in df.iterrows():
    dic = {}
    dic['id'] = index + 1
#     index_to_id[counter] = index
#     counter += 1
#     dic['amount'] = random.randint(1, 10)
    dic['amount'] = 1
    dic['latitude'] = row['lon']
    dic['longitude'] = row['lat']
    points.append(dic)
    
points

In [ ]:
distances = {}
cost_matrix = result
for i in range(len(cost_matrix)):
    distances[i + 1] = {}
    for j in range(len(cost_matrix)):
        distances[i + 1][j + 1] = cost_matrix[i][j]

distances

In [ ]:
jsf={
#     ,{'id':3,'capacity':500},{'id':4,'capacity':500},{'id':5,'capacity':500},{'id':6,'capacity':500}
"cars":[{"id":1,"capacity":5000},{'id':2,'capacity':5000}],
"constraint":{'distance':[],'time':[distances]},
"points":points,
"cost":[distances]
}
data = create_data_model(jsf)
# data = {'points': df[['lon','lat']].to_numpy(),
#  'distance_matrix': cost_matrix,
#  'codes': [1, 0],
#  'cost_matrix': cost_matrix,
#  'num_vehicles': 5,
#  'start': [0, 0],
#  'end': [0, 0],
#  'depot': 0,
#  'demands': [15] * 25,
#  'vehicle_capacities': [50, 50, 50, 50, 50]}
data

In [ ]:
def get_bearing(p1, p2):

    '''Returns compass bearing from p1 to p2
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon

    Return compass bearing of type float'''

    long_diff = np.radians(p2.lon - p1.lon)
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)

    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) - (np.sin(lat1) * np.cos(lat2) * np.cos(long_diff)))    
    bearing = np.degrees(np.arctan2(x, y))

    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing
def get_arrows(locations, color, size=6, n_arrows=3):
    '''
      Get a list of correctly placed and rotated 
      arrows/markers to be plotted

      Parameters
      locations : list of lists of lat lons that represent the 
            start and end of the line. 
            eg [[41.1132, -96.1993],[41.3810, -95.8021]]
      arrow_color : default is 'blue'
      size : default is 6
      n_arrows : number of arrows to create.  default is 3    Return
      list of arrows/markers
    '''
    Point = namedtuple('Point', field_names=['lat', 'lon'])

    p1 = Point(float(locations[0][0]), float(locations[0][1]))
    p2 = Point(float(locations[1][0]), float(locations[1][1]))

    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
#     rotation = 45.0

    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]

    arrows = []

    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                  fill_color=color, number_of_sides=3, 
                  radius=size, rotation=rotation))
    return arrows


def route_folium(result,points):

    colors = ['blue','orange','green','gray','lightgreen','darkblue','lightblue','purple','beige','pink','cadetblue','lightgray','black']
    #print(result)
    lat_long=[]
    noptimized=folium.Map(location=points[0] , zoom_start=12)
    antpath1 = plugins.AntPath(locations=points,color='red')
    antpath1.add_to(noptimized)
    folium.Marker(location = points[0] , popup='0').add_to(noptimized)
    [folium.Marker(location = points[ind],
          popup=str(ind),
          icon=folium.Icon(color='red')).add_to(noptimized) for ind in np.arange(len(points))]
    mymap= folium.Map(location=result[0][1][0] , zoom_start=12)
    for items in result:
        pointlist=items[1]
        count=items[0]
        color1=colors[count]
        folium.Marker(location =pointlist[0] , popup='0',icon=folium.Icon(color='red')).add_to(mymap)
        [folium.Marker(location = pointlist[ind] , popup=str(ind) ).add_to(mymap) for ind in range(1,len(pointlist)-1)]
        for i,l in enumerate(pointlist):
            if i==(len(pointlist)-1):
                break
            p1=pointlist[i]
            p2=pointlist[i+1] 
            folium.PolyLine(locations=[p1, p2], color=color1,weight=3, opacity=0.8).add_to(mymap)
            arrows = get_arrows(locations=[p1, p2], color=color1,n_arrows=3)
            for arrow in arrows:
                arrow.add_to(mymap)
    #return mymap
    mymap.save('after-dis-cost-first '+str(datetime.now())+'.html')
    noptimized.save('before-dis-cost-first '+str(datetime.now())+'.html')
    print('maps Saved!')

In [ ]:
route=main(data)
routes=[]
for i in range(len(route)):
    route_list=[]
    for j in range(len(route[i])):
        index=route[i][j]
        route_list.append(data['points'][index])
    rl=[i,route_list]
    routes.append(rl)
route_folium(routes,data['points'])

In [ ]:
counter = 0
for i in route[0]:
    q = """insert into missions (driver_id, latitude, longitude,state,"order",description,due_date) Values (42,%f,%f,0,%d,'%s','2020-06-07 03:30:00');""" % (df.iloc[i]['lon'],df.iloc[i]['lat'],counter,df.iloc[i]['آدرس'])
    counter += 1
    print(q)

In [ ]:
route